In [1]:
library(dplyr)
library(caret)
library(randomForest)
# id: 데이터 고유 id
# age: 나이
# sex: 성별 (여자 = 0, 남자 = 1)
# cp: 가슴 통증(chest pain) 종류 
# 0 : asymptomatic 무증상
# 1 : atypical angina 일반적이지 않은 협심증
# 2 : non-anginal pain 협심증이 아닌 통증
# 3 : typical angina 일반적인 협심증
# trestbps: (resting blood pressure) 휴식 중 혈압(mmHg)
# chol: (serum cholestoral) 혈중 콜레스테롤 (mg/dl)
# fbs: (fasting blood sugar) 공복 중 혈당 (120 mg/dl 이하일 시 = 0, 초과일 시 = 1)
# restecg: (resting electrocardiographic) 휴식 중 심전도 결과 
# 0: showing probable or definite left ventricular hypertrophy by Estes' criteria
# 1: 정상
# 2: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
# thalach: (maximum heart rate achieved) 최대 심박수
# exang: (exercise induced angina) 활동으로 인한  협심증 여부 (없음 = 0, 있음 = 1)
# oldpeak: (ST depression induced by exercise relative to rest) 휴식 대비 운동으로 인한 ST 하강
# slope: (the slope of the peak exercise ST segment) 활동 ST 분절 피크의 기울기
# 0: downsloping 하강
# 1: flat 평탄
# 2: upsloping 상승
# ca: number of major vessels colored by flouroscopy 형광 투시로 확인된 주요 혈관 수 (0~3 개) 
# Null 값은 숫자 4로 인코딩됨 
# thal: thalassemia 지중해빈혈 여부
# 0 = Null 
# 1 = normal 정상
# 2 = fixed defect 고정 결함
# 3 = reversable defect 가역 결함
# target: 심장 질환 진단 여부
# 0: < 50% diameter narrowing
# 1: > 50% diameter narrowing



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:ggplot2’:

    margin

The following object is masked from ‘package:dplyr’:

    combine



In [2]:
heart=read.csv("../Data/dataset/train.csv")

In [3]:
heart %>%str 

'data.frame':	151 obs. of  15 variables:
 $ id      : int  1 2 3 4 5 6 7 8 9 10 ...
 $ age     : int  53 52 54 45 35 49 41 60 55 35 ...
 $ sex     : int  1 1 1 0 1 1 1 1 1 0 ...
 $ cp      : int  2 3 1 0 1 2 1 2 0 0 ...
 $ trestbps: int  130 152 192 138 122 120 110 140 140 138 ...
 $ chol    : int  197 298 283 236 192 188 235 185 217 183 ...
 $ fbs     : int  1 1 0 0 0 0 0 0 0 0 ...
 $ restecg : int  0 1 0 0 1 1 1 0 1 1 ...
 $ thalach : int  152 178 195 152 174 139 153 155 111 182 ...
 $ exang   : int  0 0 0 1 0 0 0 0 1 0 ...
 $ oldpeak : num  1.2 1.2 0 0.2 0 2 0 3 5.6 1.4 ...
 $ slope   : int  0 1 2 1 2 1 2 1 0 2 ...
 $ ca      : int  0 0 1 0 0 3 0 0 0 0 ...
 $ thal    : int  2 3 3 2 2 3 2 2 3 2 ...
 $ target  : int  1 1 0 1 1 0 1 0 0 1 ...


In [4]:
heart %>% filter(target==1)%>% nrow#걸린사람
heart %>% filter(target!=1)%>% nrow#안걸린사람

[1] 83

[1] 68

In [5]:
# one hot 해야하는거 sex cp restecg exang slope thal
heart

id,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
1,53,1,2,130,197,1,0,152,0,1.2,0,0,2,1
2,52,1,3,152,298,1,1,178,0,1.2,1,0,3,1
3,54,1,1,192,283,0,0,195,0,0.0,2,1,3,0
4,45,0,0,138,236,0,0,152,1,0.2,1,0,2,1
5,35,1,1,122,192,0,1,174,0,0.0,2,0,2,1
6,49,1,2,120,188,0,1,139,0,2.0,1,3,3,0
7,41,1,1,110,235,0,1,153,0,0.0,2,0,2,1
8,60,1,2,140,185,0,0,155,0,3.0,1,0,2,0
9,55,1,0,140,217,0,1,111,1,5.6,0,0,3,0
10,35,0,0,138,183,0,1,182,0,1.4,2,0,2,1


In [6]:
cor(heart)[,15] # 타겟과의 상관계수 

id         age         sex          cp    trestbps        chol 
-0.04488973 -0.24780599 -0.24628874  0.43627345 -0.12285039  0.03699056 
        fbs     restecg     thalach       exang     oldpeak       slope 
 0.06582057  0.01752795  0.36524886 -0.39614493 -0.42151430  0.30599382 
         ca        thal      target 
-0.46628934 -0.42853028  1.00000000

In [7]:
#train_df_T$Embarked_C = train_df_T$Embarked == "C"
heart$sex_M=heart$sex == 1
heart$sex_F=heart$sex != 1
heart$cp_0=heart$cp==0
heart$cp_1=heart$cp==1
heart$cp_2=heart$cp==2
heart$cp_3=heart$cp==3
heart$restecg_0=heart$restecg==0
heart$restecg_1=heart$restecg==1
heart$restecg_2=heart$restecg==2
heart$exang_0=heart$exang==0
heart$exang_1=heart$exang==1
heart$slope_0=heart$slope==0
heart$slope_1=heart$slope==1
heart$slope_2=heart$slope==2
heart$thal_0=heart$thal==0
heart$thal_1=heart$thal==1
heart$thal_2=heart$thal==2
heart$thal_3=heart$thal==3

In [8]:
heart$cp_0=heart$cp==0
heart$cp_1=heart$cp==1
heart$cp_2=heart$cp==2
heart$cp_3=heart$cp==3

In [9]:
heart$restecg_0=heart$restecg==0
heart$restecg_1=heart$restecg==1
heart$restecg_2=heart$restecg==2

In [10]:
heart$exang_0=heart$exang==0
heart$exang_1=heart$exang==1

In [11]:
heart$oldpeak %>%summary
heart$oldpeak %>% table

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0000  0.0000  0.8000  0.9768  1.6000  5.6000 

.
  0 0.1 0.2 0.3 0.4 0.5 0.6 0.8 0.9   1 1.2 1.4 1.5 1.6 1.8 1.9   2 2.2 2.3 2.4 
 53   3   3   1   4   4   6   9   1   8  12   5   3   7   4   2   2   3   2   1 
2.6 2.8 2.9   3 3.1 3.4 3.5 3.8   4 5.6 
  4   4   1   2   1   2   1   1   1   1 

In [12]:
heart$old_0=heart$oldpeak==0
heart$old_1=heart$oldpeak>0 & heart$oldpeak <=0.8
heart$old_2=heart$oldpeak>0.8 & heart$oldpeak <=1.6
heart$old_3=heart$oldpeak>1.6 

In [13]:
heart

id,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,...,slope_1,slope_2,thal_0,thal_1,thal_2,thal_3,old_0,old_1,old_2,old_3
1,53,1,2,130,197,1,0,152,0,...,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE
2,52,1,3,152,298,1,1,178,0,...,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE
3,54,1,1,192,283,0,0,195,0,...,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE
4,45,0,0,138,236,0,0,152,1,...,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE
5,35,1,1,122,192,0,1,174,0,...,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE
6,49,1,2,120,188,0,1,139,0,...,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE
7,41,1,1,110,235,0,1,153,0,...,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE
8,60,1,2,140,185,0,0,155,0,...,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE
9,55,1,0,140,217,0,1,111,1,...,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE
10,35,0,0,138,183,0,1,182,0,...,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE


In [14]:
heart$slope_0=heart$slope==0
heart$slope_1=heart$slope==1
heart$slope_2=heart$slope==2

In [15]:
heart$thal_0=heart$thal==0
heart$thal_1=heart$thal==1
heart$thal_2=heart$thal==2
heart$thal_3=heart$thal==3

In [16]:
heart%>%colnames

[1] "id"        "age"       "sex"       "cp"        "trestbps"  "chol"     
 [7] "fbs"       "restecg"   "thalach"   "exang"     "oldpeak"   "slope"    
[13] "ca"        "thal"      "target"    "sex_M"     "sex_F"     "cp_0"     
[19] "cp_1"      "cp_2"      "cp_3"      "restecg_0" "restecg_1" "restecg_2"
[25] "exang_0"   "exang_1"   "slope_0"   "slope_1"   "slope_2"   "thal_0"   
[31] "thal_1"    "thal_2"    "thal_3"    "old_0"     "old_1"     "old_2"    
[37] "old_3"

In [17]:
heart=heart%>%arrange(target)

In [18]:
heart
#sex cp restecg exang slope thal
heart$target=heart$target%>%as.factor

id,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,...,slope_1,slope_2,thal_0,thal_1,thal_2,thal_3,old_0,old_1,old_2,old_3
3,54,1,1,192,283,0,0,195,0,...,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE
6,49,1,2,120,188,0,1,139,0,...,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE
8,60,1,2,140,185,0,0,155,0,...,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE
9,55,1,0,140,217,0,1,111,1,...,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE
11,68,1,2,180,274,1,0,150,1,...,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE
13,65,1,0,135,254,0,0,127,0,...,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE
19,59,1,0,170,326,0,0,140,1,...,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE
21,62,1,1,120,281,0,0,103,0,...,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE
24,61,0,0,145,307,0,0,146,1,...,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE
25,70,1,0,145,174,0,1,125,1,...,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE


In [19]:
set.seed(1199)
samp1 <- createDataPartition(heart$target, p = 0.7, list = F)
target_col = c("target","age","trestbps","chol","fbs","thalach","oldpeak","ca",
              'sex_M', 'sex_F' ,'cp_0', 'cp_1', 'cp_2' ,'cp_3', 
               'exang_0', 'exang_1' , 'slope_0' ,'slope_1', 'slope_2'
               , 'thal_0', 'thal_1', 'thal_2', 'thal_3')

data.train2 <- heart[samp1,]
#검즘용
data.test2 <- heart[-samp1,]
train.data2 = data.train2[target_col]
test.data2 = data.test2[target_col]
rf= randomForest(target ~ . , data= train.data2 ,mtry = 40,importance = T, ntree = 2000)
x = subset(test.data2,select = -target)
y = test.data2$target
pred = predict(rf,x)

Warning message in randomForest.default(m, y, ...):
“invalid mtry: reset to within valid range”

In [20]:
rr=pred

In [21]:
test=read.csv("..//Data/dataset/test.csv")

In [22]:
test$sex_M=test$sex == 1
test$sex_F=test$sex != 1
test$cp_0=test$cp==0
test$cp_1=test$cp==1
test$cp_2=test$cp==2
test$cp_3=test$cp==3
test$restecg_0=test$restecg==0
test$restecg_1=test$restecg==1
test$restecg_2=test$restecg==2
test$exang_0=test$exang==0
test$exang_1=test$exang==1
test$slope_0=test$slope==0
test$slope_1=test$slope==1
test$slope_2=test$slope==2
test$thal_0=test$thal==0
test$thal_1=test$thal==1
test$thal_2=test$thal==2
test$thal_3=test$thal==3

In [23]:
set.seed(1120)

target_col = c("target","age","trestbps","chol","fbs","thalach","oldpeak","ca",
              'sex_M', 'sex_F' ,'cp_0', 'cp_1', 'cp_2' ,'cp_3', 'restecg_0' ,'restecg_1', 'restecg_2',
               'exang_0', 'exang_1' , 'slope_0' ,'slope_1', 'slope_2'
               , 'thal_0', 'thal_1', 'thal_2', 'thal_3')

data.train2 <- heart
#검즘용
data.test2 <- heart
train.data2 = data.train2[target_col]
test.data2 = data.test2[target_col]
rf= randomForest(target ~ . , data= train.data2 ,mtry = 40,importance = T, ntree = 2000)
x = subset(test.data2,select = -target)
y = test.data2$target
pred = predict(rf,x)

Warning message in randomForest.default(m, y, ...):
“invalid mtry: reset to within valid range”

In [24]:
mean(pred==y)
table(pred,y)

[1] 1

    y
pred  0  1
   0 68  0
   1  0 83

In [25]:
rr=predict(rf,test)



In [26]:
result=read.csv("../Data/dataset/sample_submission.csv")

In [27]:
result$target=rr
result

id,target
1,0
2,1
3,1
4,0
5,1
6,1
7,1
8,0
9,0
10,1


In [28]:
write.csv(
  result,              # 파일에 저장할 데이터 프레임 또는 행렬
  file="result2.csv",        # 데이터를 저장할 파일명
  row.names=F  # TRUE면 행 이름을 CSV 파일에 포함하여 저장한다.
)

In [29]:
read.csv("..//Note/result.csv")

id,target
1,0
2,1
3,1
4,0
5,1
6,1
7,1
8,0
9,0
10,1


In [30]:
read.csv("..//Note/result2.csv")

id,target
1,0
2,1
3,1
4,0
5,1
6,1
7,1
8,0
9,0
10,1


In [32]:
study=read.csv("../../../Downloads/loan-default-prediction/train_v2.csv")